In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Cystic_Fibrosis"

# Input paths
tcga_root_dir = "../../input/TCGA"

# Output paths
out_data_file = "../../output/preprocess/Cystic_Fibrosis/TCGA.csv"
out_gene_data_file = "../../output/preprocess/Cystic_Fibrosis/gene_data/TCGA.csv"
out_clinical_data_file = "../../output/preprocess/Cystic_Fibrosis/clinical_data/TCGA.csv"
json_path = "../../output/preprocess/Cystic_Fibrosis/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
import os
import pandas as pd

# 1. Find the most relevant directory for Colon and Rectal Cancer
subdirectories = os.listdir(tcga_root_dir)
target_trait = trait.lower().replace("_", " ")  # Convert to lowercase for case-insensitive matching

# Start with no match, then find the best match based on similarity to target trait
best_match = None
best_match_score = 0

for subdir in subdirectories:
    subdir_lower = subdir.lower()
    
    # Calculate a simple similarity score - more matching words = better match
    # This prioritizes exact matches over partial matches
    score = 0
    for word in target_trait.split():
        if word in subdir_lower:
            score += 1
    
    # Track the best match
    if score > best_match_score:
        best_match_score = score
        best_match = subdir
        print(f"Found potential match: {subdir} (score: {score})")

# Use the best match if found
if best_match:
    print(f"Selected directory: {best_match}")
    
    # 2. Get the clinical and genetic data file paths
    cohort_dir = os.path.join(tcga_root_dir, best_match)
    clinical_file_path, genetic_file_path = tcga_get_relevant_filepaths(cohort_dir)
    
    print(f"Clinical file: {os.path.basename(clinical_file_path)}")
    print(f"Genetic file: {os.path.basename(genetic_file_path)}")
    
    # 3. Load the data files
    clinical_df = pd.read_csv(clinical_file_path, sep='\t', index_col=0)
    genetic_df = pd.read_csv(genetic_file_path, sep='\t', index_col=0)
    
    # 4. Print clinical data columns for inspection
    print("\nClinical data columns:")
    print(clinical_df.columns.tolist())
    
    # Print basic information about the datasets
    print(f"\nClinical data shape: {clinical_df.shape}")
    print(f"Genetic data shape: {genetic_df.shape}")
    
    # Check if we have both gene and trait data
    is_gene_available = genetic_df.shape[0] > 0
    is_trait_available = clinical_df.shape[0] > 0
    
else:
    print(f"No suitable directory found for {trait}.")
    is_gene_available = False
    is_trait_available = False

# Record the data availability
validate_and_save_cohort_info(
    is_final=False,
    cohort="TCGA",
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# Exit if no suitable directory was found
if not best_match:
    print("Skipping this trait as no suitable data was found.")

No suitable directory found for Cystic_Fibrosis.
Skipping this trait as no suitable data was found.
